<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Final (Dashboard tema libre)**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

Verónica Marisela Frías Peralta (208561486)

[veronica.frias6148@alumnos.udg.mx](mailto:veronica.frias6148@alumnos.udg.mx)

**Requisitos mínimos:**
* **Capa 1**: Un panel de controles que permita generar el **dataset origen** a analizar. Este dataset puede ser generado al momento partiendo de una formula e introduciendo los parametros a travez de los widgets en el panel (**ejemplo**: selección del tipo de trayectoria y sus parámetros en la práctica 4), o puede ser el resultado de filtrar un dataset mayor ya existente en memoria, en cuyo caso los widgets en el panel de control permitiran ajustar los limites de dichos filtros.
* **Capa 2**: Un panel en el cual se despliegue una **representación gráfica del dataset origen** (**ejemplo**: Trazo de la trayectoria en 3D en la práctica 4).
* **Capa 3**: Un panel de controles que permita elegir entre diferentes **metricas** a calcular a partir del dataset origen. Las métricas obtenidas deberán ser representadas también graficamente (**ejemplo**: Selección entre path length y mean squared displacement y su gráfica en la práctica 4).

**Revisión de la práctica:**
* El notebook con la práctica se entregará en un repositorio en **GitHub**. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* Los integrantes del equipo deberán explicar su código y deberán ser capaces de corregir errores introducidos a este.
* Uno de los integrantes del equipo presentará el dashboard durante la clase.

## **Modules**

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
import plotly.express as px
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance
from google.colab import drive
pn.extension('plotly')

In [32]:
# Crear una función para generar el dataset según una fórmula
def generar_dataset(formula_param1, formula_param2, size=100):
    np.random.seed(42)
    x = np.linspace(0, 10, size)
    y = formula_param1 * x + formula_param2 + np.random.normal(scale=2, size=size)

    dataset = pd.DataFrame({'X': x, 'Y': y})
    return dataset

# Crear widgets para ajustar los parámetros de la fórmula
formula_param1_widget = pn.widgets.FloatSlider(name='Fórmula Parametro 1', start=0, end=10, value=5)
formula_param2_widget = pn.widgets.FloatSlider(name='Fórmula Parametro 2', start=0, end=10, value=5)

# Crear widget de salida para mostrar información del dataset generado
dataset_output = pn.widgets.DataFrame()

# Función para actualizar el dataset según los parámetros seleccionados
def actualizar_dataset(event):
    formula_param1 = formula_param1_widget.value
    formula_param2 = formula_param2_widget.value

    dataset = generar_dataset(formula_param1, formula_param2)
    dataset_output.value = dataset

# Conectar widgets a la función de actualización
formula_param1_widget.link(actualizar_dataset, value='value')
formula_param2_widget.link(actualizar_dataset, value='value')

# Crear el panel de controles
panel_controles = pn.Column(
    '## Capa 1: Panel de Controles',
    'Ajuste los parámetros para generar el dataset:',
    formula_param1_widget,
    formula_param2_widget
)

# Crear el dashboard que incluye el panel de controles y el widget de salida para mostrar el dataset
dashboard = pn.Column(
    panel_controles,
    '## Dataset Generado',
    dataset_output
)

# Mostrar el dashboard
dashboard.servable()

Column
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] FloatSlider(end=10, name='Fórmula Parametro 1', value=5)
        [3] FloatSlider(end=10, name='Fórmula Parametro 2', value=5)
    [1] Markdown(str)
    [2] DataFrame()

In [ ]:
# Cargar el conjunto de datos
url = '/content/conjunto_de_datos_gastotarjetas_enigh2022_ns.csv'
df = pd.read_csv(url)

# Obtener las categorías únicas para la columna 'clave'
categorias = df['clave'].unique().tolist()  # Convertir a lista

# Crear widgets para seleccionar las categorías y visualizar gráficamente
categoria_widget = pn.widgets.Select(options=categorias, name='Seleccione una categoría')
grafico_output = pn.pane.Matplotlib(width=800, height=400)

# Función para actualizar el gráfico según la categoría seleccionada
def actualizar_grafico(event):
    categoria_seleccionada = categoria_widget.value
    datos_filtrados = df[df['clave'] == categoria_seleccionada]

    plt.figure(figsize=(10, 6))
    plt.bar(datos_filtrados['folioviv'], datos_filtrados['gasto'])
    plt.xlabel('Folio de Vivienda')
    plt.ylabel('Gasto')
    plt.title(f'Gasto por Vivienda para la Categoría: {categoria_seleccionada}')
    plt.xticks(rotation=45)

    grafico_output.object = plt.gcf()

# Conectar el widget a la función de actualización
categoria_widget.link(actualizar_grafico, value='value')

# Crear y mostrar el panel
panel = pn.Column(
    '## Visualización de Gastos por Categoría',
    categoria_widget,
    '### Gráfico:',
    grafico_output
)

panel.servable()

In [30]:
import pandas as pd
import panel as pn
import matplotlib.pyplot as plt

# Cargar el conjunto de datos
url = '/content/conjunto_de_datos_gastotarjetas_enigh2022_ns.csv'
df = pd.read_csv(url)

# Obtener las columnas únicas para los widgets de selección
columnas_numericas = df.select_dtypes(include='number').columns.tolist()
columnas_categoricas = df.select_dtypes(include='object').columns.tolist()

# Crear widgets para seleccionar las columnas
columna_gasto_widget = pn.widgets.Select(options=columnas_numericas, name='Seleccione la columna de gasto')
columna_foliohog_widget = pn.widgets.Select(options=columnas_numericas, name='Seleccione la columna de foliohog')
columna_clave_widget = pn.widgets.Select(options=columnas_categoricas, name='Seleccione la columna de clave')

# Crear widget de salida para mostrar el gráfico
grafico_output = pn.pane.Matplotlib(width=800, height=400)

# Función para actualizar el gráfico según las columnas seleccionadas
def actualizar_grafico(event):
    columna_gasto = columna_gasto_widget.value
    columna_foliohog = columna_foliohog_widget.value
    columna_clave = columna_clave_widget.value

    plt.figure(figsize=(10, 6))
    df_grouped = df.groupby([columna_clave, columna_foliohog])[columna_gasto].sum().unstack()
    df_grouped.plot(kind='bar', stacked=True, ax=plt.gca())
    plt.xlabel(columna_clave)
    plt.ylabel(columna_gasto)
    plt.title(f'Gasto por {columna_clave} y {columna_foliohog}')
    plt.legend(title=columna_foliohog)

    grafico_output.object = plt.gcf()

# Conectar widgets a la función de actualización
columna_gasto_widget.link(actualizar_grafico, value='value')
columna_foliohog_widget.link(actualizar_grafico, value='value')
columna_clave_widget.link(actualizar_grafico, value='value')

# Crear y mostrar el panel
panel = pn.Column(
    '## Visualización de Gastos por Columnas',
    'Seleccione las columnas para visualizar:',
    columna_gasto_widget,
    columna_foliohog_widget,
    columna_clave_widget,
    '### Gráfico:',
    grafico_output
)

panel.servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Select(name='Seleccione la c..., options=['folioviv', 'foliohog', ...], value='folioviv')
    [3] Select(name='Seleccione la c..., options=['folioviv', 'foliohog', ...], value='folioviv')
    [4] Select(name='Seleccione la c..., options=['clave', 'pago_mp'], value='clave')
    [5] Markdown(str)
    [6] Matplotlib(None, height=400, sizing_mode='fixed', width=800)